In [6]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing, cross_validation, svm, neighbors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

teamsDF = pd.read_csv('../data/Team.csv')
matchesDF = pd.read_csv('../data/Match.csv')

def getMatchesForTeam(id):
    return matchesDF[(matchesDF["Team_Name_Id"] == id) | (matchesDF["Opponent_Team_Id"] == id)]

def getMatchesWonByTeam(id):
    return matchesDF[matchesDF["Match_Winner_Id"] == id]


def getTeamWinPercentage(id):
    result = 0
    totalMatches = len(getMatchesForTeam(id))
    matchesWon = len(getMatchesWonByTeam(id))
    if(totalMatches>0):
        result = (matchesWon*100)/totalMatches
    return result
    
def generatePredictData(teamId,opponentId,tossWon,batFirst):
    data = pd.DataFrame(columns=('Match_Id','Team_Id','Toss_Won','Bat_First','Win_Percenetage','Opponent_Win_Percentage','Win_Percenetage_Against','Toss_Decision_Win_Percentage','Bat_Decision_Win_Percentage','Match_Won'))
    pos = 0
    matchId = 1
    teamTossWon = 0
    teamBatFirst = 0
    teamWinPercentage = getTeamWinPercentage(teamId)
    opponentTeamWinPercentage = getTeamWinPercentage(opponentId)
    teamMatchWinPercentAgainstOpponent = getMatchWonPercentageForTeamAgainstTeam(teamId,opponentId)
    teamTossDescionWinPercentage = getMatchWonPercentWhenTossWonForTeam(teamId)
    teamBatDecisionWonPercentage = getFirstBatWonPercentage(teamId)
    if(tossWon == teamId):
        teamTossWon = 1
    else:
        teamTossDescionWinPercentage = 100 - teamTossDescionWinPercentage

    if(batFirst == teamId):
        teamBatFirst = 1
    else:
        teamBatDecisionWonPercentage = 100 - teamBatDecisionWonPercentage
    data.loc[pos,"Match_Id"] = matchId
    data.loc[pos,"Team_Id"] = teamId
    data.loc[pos,"Toss_Won"] = teamTossWon
    data.loc[pos,"Bat_First"] = teamBatFirst
    data.loc[pos,"Win_Percenetage"] = teamWinPercentage
    data.loc[pos,"Opponent_Win_Percentage"] = opponentTeamWinPercentage
    data.loc[pos,"Win_Percenetage_Against"] = teamMatchWinPercentAgainstOpponent
    data.loc[pos,"Toss_Decision_Win_Percentage"] = teamTossDescionWinPercentage
    data.loc[pos,"Bat_Decision_Win_Percentage"] = teamBatDecisionWonPercentage
    data.loc[pos,"Match_Won"] = 0
    return data

matchesTeamsDF = pd.read_csv("../data/generate/match_team.csv")
data = matchesTeamsDF.drop(['Match_Id'],1)
data = data.drop(['Unnamed: 0'],1)
X = data.drop(['Match_Won'],1)
Y = data['Match_Won']

X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size = 0.25)

def predict():
    print("Enter Team A Id")
    teamId = input()
    while(len(teamsDF[teamsDF["Team_Id"] == int(teamId)]) == 0):
        print("Please Enter valid Team Id")
        teamId = input()
    print("Enter Team B Id")
    opponentId = input()
    while(len(teamsDF[teamsDF["Team_Id"] == int(opponentId)]) == 0):
        print("Please Enter valid Team Id")
        opponentId = input()
    print("Which team won the toss?Enter Id")
    tossWon = input()
    while((tossWon != teamId) & (tossWon != opponentId)):
        print("Please Enter valid Team Id. %s or %s" % (teamId,opponentId))
        tossWon = input()
    print("Which team bat first?Enter Id")
    batFirst = input()
    while((batFirst != teamId) & (batFirst != opponentId)):
        print("Please Enter valid Team Id. %s or %s" % (teamId,opponentId))
        batFirst = input()
    px = generatePredictData(int(teamId),int(opponentId),int(tossWon),int(batFirst))
    px = px.drop(['Match_Id'],1)
    px = px.drop(['Match_Won'],1)

    px = preprocessing.scale(px)

    lin_svm = svm.LinearSVC()
    lin_svm.fit(X_train, y_train)
    pred = lin_svm.predict(px)
    if(pred[0] == 1):
        print("Team A Wins")
    else:
        print("Team B Wins")


predict()


Enter Team A Id
1
Enter Team B Id
7
Which team won the toss?Enter Id
7
Which team bat first?Enter Id
1


NameError: name 'getMatchWonPercentageForTeamAgainstTeam' is not defined